In [28]:
import MySQLdb
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import sys
import cb104_addr

# "CT" "apple" "storm" "udn"
paper_name = "apple"

mongodb_db_host = cb104_addr.local_mongodb_host
mongodb_db_name = "keyword_filter"
mongodb_collection = paper_name + "_news"

mariadb_host = cb104_addr.local_mariadb_host
mariadb_user = cb104_addr.local_mariadb_user
mariadb_pw = cb104_addr.local_mariadb_pw
mariadb_db_name = "news_db"
mariadb_tablename = paper_name + "_news"

conn_md = MongoClient(mongodb_db_host)
mdb = conn_md[mongodb_db_name]
mcollection = mdb[mongodb_collection]

conn = MySQLdb.connect(host=mariadb_host, user=mariadb_user, passwd=mariadb_pw, db=mariadb_db_name, charset="utf8")
cursor = conn.cursor()

sql_command = "select create_time,tag,title_keyword_1,title_keyword_2,title_keyword_3,keyword_1,keyword_2,keyword_3," \
              "keyword_4,keyword_5,views_48 from " + mariadb_tablename + \
              " where views_48 is not null and create_time > '2019-02-07'"
cursor.execute(sql_command)
sql_data = cursor.fetchall()
cursor.close()
conn.close()

print(len(sql_data))


def take_second(elem):
    return elem[1]


# 讀取keyword資料
keywordDict = {}
count1 = 0
with open("new_keyword0317.txt","r",encoding="utf-8") as kwh:
    data = kwh.read()
    data_line = data.split("\n")
    kwSet = set()
    for i in range(1, len(data_line)-1):
        # print("data",data_line[i])
        a = eval(data_line[i])

        if a[0].strip().upper() not in keywordDict:
            keywordDict[a[0].strip().upper()] = a[1]
        else:
            keywordDict[a[0].strip().upper()] = keywordDict[a[0].strip().upper()] + a[1]
delList = []
for keyword in keywordDict:
    if keywordDict[keyword] < 100:
        delList.append(keyword)

for keyword in delList:
    del keywordDict[keyword]

for i in range(10,21):
    if str(i) in keywordDict:
        del keywordDict[str(i)]
title = ["k1_24", "k1_72", "k1_168", "k1_trend",
         "k2_24", "k2_72", "k2_168", "k2_trend",
         "k3_24", "k3_72", "k3_168", "k3_trend", "48hr_views","tag"]
df_news = pd.DataFrame(columns=title)

count_data = 0
for sqlDataOneRow in sql_data:
    newsCreateTime = sqlDataOneRow[0]
    # print(newsCreateTime)
    titleCount = 0
    dfData = {}
    keywordData = set()
    # 這邊要判斷有沒有進重要字
    for keywordPos in range(2, 10):
        if sqlDataOneRow[keywordPos] is not None:
            keywordData.add(sqlDataOneRow[keywordPos].strip().upper())

    sqlKeywordList = []
    for keyword in keywordData:
        if keyword in keywordDict:
            sqlKeywordList.append([keyword, keywordDict[keyword]])

    if len(sqlKeywordList) >= 3:
        count1 = count1 + 1
        sqlKeywordList.sort(key=take_second, reverse=True)
        # print(sqlKeywordList)
    else:
        sqlKeywordList.append(None)
        sqlKeywordList.append(None)
        sqlKeywordList.append(None)
    count_data = count_data + 1
    if count_data % 500 == 0:
        print("已處理資料:",count_data)
    keyName = ["k1", "k2", "k3"]
    try:
        for keywordPos in range(3):
            if sqlKeywordList[keywordPos] is None:
                dfData[keyName[keywordPos]+"_24"] = 0
                dfData[keyName[keywordPos]+"_72"] = 0
                dfData[keyName[keywordPos]+"_168"] = 0
                dfData[keyName[keywordPos] + "_trend"] = 0
                continue
            # views = [[0],[0],[0],[0],[0],[0],[0]]
            views = [0,0,0,0,0,0,0]
            m_data = mcollection.find({"keyword": sqlKeywordList[keywordPos][0]})[0]
            # print(m_data)
            for viewsCreateTimeStr in m_data["views"]:
                viewsCreateTime = datetime.strptime(viewsCreateTimeStr, "%Y-%m-%d %H:%M")
                dayDiff = (newsCreateTime - viewsCreateTime).total_seconds() // 86400
                dayDiff = int(dayDiff)
                if 0 <= dayDiff < 7:
                    # print(viewsCreateTime, newsCreateTime, dayDiff, m_data["views"][viewsCreateTimeStr])
                    views[dayDiff] = views[dayDiff] + int(m_data["views"][viewsCreateTimeStr])
                dfData[keyName[keywordPos]+"_24"] = views[0]
                dfData[keyName[keywordPos]+"_72"] = views[1] + views[2]
                dfData[keyName[keywordPos]+"_168"] = views[3] + views[4] + views[5] + views[6]
                # 計算斜率
                # model = LinearRegression(fit_intercept=True)
                lm = LinearRegression()
                # print(views)
                lm.fit(np.array([[0], [-1], [-2], [-3], [-4], [-5], [-6]]), np.array(views))
                # print(lm.intercept_)
                # print(lm.coef_)
                dfData[keyName[keywordPos] + "_trend"] = lm.coef_
        dfData["48hr_views"] = sqlDataOneRow[-1]
        dfData["tag"] = sqlDataOneRow[1]
        df_news = df_news.append(dfData, ignore_index=True)
    except Exception as e:
        print(sqlKeywordList)
        print(e)

15992
已處理資料: 500
已處理資料: 1000
已處理資料: 1500
已處理資料: 2000
已處理資料: 2500
已處理資料: 3000
已處理資料: 3500
已處理資料: 4000
已處理資料: 4500
已處理資料: 5000
已處理資料: 5500
已處理資料: 6000
已處理資料: 6500
已處理資料: 7000
已處理資料: 7500
已處理資料: 8000
已處理資料: 8500
已處理資料: 9000
已處理資料: 9500
已處理資料: 10000
已處理資料: 10500
已處理資料: 11000
已處理資料: 11500
已處理資料: 12000
已處理資料: 12500
已處理資料: 13000
已處理資料: 13500
已處理資料: 14000
已處理資料: 14500
已處理資料: 15000
已處理資料: 15500


In [29]:
df_news.to_pickle("df_news_3keyword_puls0.pkl")

In [3]:
def change48VToClass(num):
    if num <= 1000:
        return "一千以下"
    elif num <= 10000:
        return "一萬以下"
    elif num <= 100000:
        return "十萬以下"
    elif num <= 1000000:
        return "百萬以下"
    else:
        return "超過百萬"

print(max(list(df_news["48hr_views"])))
    
df_news_cls = df_news
df_news_cls["class"] = df_news["48hr_views"].apply(change48VToClass)

a = set()
for tmp in list(df_news_cls["class"]):
    a.add(tmp)
print(a)


1555814
{'一千以下', '百萬以下', '十萬以下', '超過百萬', '一萬以下'}


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

x_total = df_news_cls.drop(["class","tag","48hr_views","k1_72","k2_72","k3_72"],axis = 1)
y_total = df_news_cls["class"]

x_train,x_test,y_train,y_test = train_test_split(x_total,y_total,test_size = 0.2)


clf = RandomForestClassifier(n_estimators=26, max_depth= 6)

# np.average(cross_val_score(clf, x_train, y_train, cv=10))
clf = clf.fit(x_train,y_train)

In [10]:
from sklearn.metrics import accuracy_score
pred = clf.predict(x_test)
print("正確率:",accuracy_score(y_test,pred))

正確率: 0.6323851203501094


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

x_total = df_news.drop(["class","tag","48hr_views","k1_72","k2_72","k3_72"],axis = 1)
y_total = df_news["48hr_views"]

x_train,x_test,y_train,y_test = train_test_split(x_total,y_total,test_size = 0.2)


clf = RandomForestRegressor(n_estimators=26, max_depth= 6)

# np.average(cross_val_score(clf, x_train, y_train, cv=10))


In [27]:
from sklearn.metrics import r2_score
clf = clf.fit(x_train,y_train)
pred = clf.predict(x_test)
print("r2score:",r2_score(y_test,pred))

r2score: 0.14170315088772822


In [25]:
x_test.isna().sum()

k1_24       0
k1_168      0
k1_trend    0
k2_24       0
k2_168      0
k2_trend    0
k3_24       0
k3_168      0
k3_trend    0
k1_48       0
k2_48       0
k3_48       0
dtype: int64